In [ ]:
### For automatically reloading import modules... allows you to run changes to code in jupyter without having to reload
%load_ext autoreload
%autoreload 2

In [ ]:
# import zipfile
# import h5py
import os
import sys
import time
import timeit
import pickle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True

from photocatalysis.learners_treesearch import get_population_completed
from photocatalysis.deeplearning.helpers import get_charset, smiles_to_onehot

In [ ]:
import torch
import torch.optim as optim
import torchinfo

In [ ]:
df = pd.read_json('/home/btpq/bt308495/Thesis/frames/DF_COMPLETE.json', orient='split')
df = get_population_completed(df)

In [ ]:
input_size = 80
char_list, max_smi_len = get_charset(df.molecule_smiles)
data = smiles_to_onehot(df.molecule_smiles, char_list, input_size)

In [ ]:
# 80/5/15 Train/Val/Test Split
data_train, data_test = train_test_split(data, test_size=0.2, shuffle=False)
data_valid, data_test = train_test_split(data_test, test_size=0.75, shuffle=False)

In [ ]:
data_train_tensor = torch.from_numpy(data_train)[:1000]
data_valid_tensor = torch.from_numpy(data_valid)[:1000]
data_test_tensor = torch.from_numpy(data_test)

# data_train_tensor_loader = torch.utils.data.TensorDataset(data_train_tensor)
train_loader = torch.utils.data.DataLoader(data_train_tensor, batch_size=250, shuffle=True)
valid_loader = torch.utils.data.DataLoader(data_valid_tensor, batch_size=250, shuffle=False)
test_loader = torch.utils.data.DataLoader(data_test_tensor, batch_size=250, shuffle=False)

-----

In [ ]:
from photocatalysis.deeplearning.models import VAE, train_epoch

In [ ]:
torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 120

# NEW MODEL
model = VAE(INPUT_SIZE=input_size, CHARSET_LEN=len(char_list), LATENT_DIM=292).to(device)

# LOAD PREV MODEL
# LOAD PREV MODEL
# model = VAE()
# model.load_state_dict(torch.load('/content/drive/MyDrive/VAE_model_parmas.pt', map_location=torch.device(device)))
# model.to(device)

optimizer = optim.Adam(model.parameters())

In [ ]:
torchinfo.summary(VAE(), input_size=(batch_size, 120, 33))

In [ ]:
len(char_list)

In [ ]:
batch_size = 1
# torchinfo.summary(VAE(), input_size=(batch_size, 120, 33))
torchinfo.summary(VAE(INPUT_SIZE=input_size, CHARSET_LEN=len(char_list)), input_size=(batch_size, input_size, len(char_list)))

In [ ]:
tls, vls = [], []
for epoch in range(1, epochs+1):
    training_losses, validation_loss = train_epoch(train_loader,
                                                   model,
                                                   optimizer,
                                                   validation_data_loader=None,
                                                   device=device,
                                                   charset=char_list,
                                                   epoch=epoch)
    
    tls.append(training_losses), vls.append(validation_loss)
    break